In [1]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [2]:
!pip install sentencepiece
!git clone https://github.com/google-research/bert



import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
# import sentencepiece as spm

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")

from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder

auth.authenticate_user()

# configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s :  %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]


if 'COLAB_TPU_ADDR' in os.environ:
  log.info("Using TPU runtime")
  USE_TPU = True
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(TPU_ADDRESS) as session:
    log.info('TPU address is ' + TPU_ADDRESS)
    # Upload credentials to TPU.
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    
else:
  log.warning('Not connected to TPU runtime')
  USE_TPU = False

drive.mount('/content/gdrive')
#!ls "/content/drive/My drive"

     |████████████████████████████████| 1.0MB 3.3MB/s 
Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 300.28 KiB | 3.95 MiB/s, done.
Resolving deltas: 100% (185/185), done.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



2020-05-11 16:38:08,838 :  Using TPU runtime
2020-05-11 16:38:08,840 :  TPU address is grpc://10.31.86.90:8470


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
file_id = {'id':"1agaKgjPrqduNoLl7teW3u7A4N8jSr0Ws"} # the id can be found in drive, in the sharing link.
file_name = "reddit_corpus.tar.gz"
extracted_file_name = "reddit"

WIKI_WORDS_IN_VOCAB = 100

SUBSAMPLED_DATA_FPATH = "subsampled.txt"

### Importing the Reddit Dataset

In [16]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile(file_id)   # replace the id with id of file you want to access
downloaded.GetContentFile(file_name)        # replace the file name with your file

!tar -xf $file_name # Unzipping
!ls $extracted_file_name | wc -l #Should be 118059

118059


#### Combining the files into a single file
Change `CORPUS_SIZE` as per requirement. It is the number of lines to be included in the final file.

`DEMO_MODE` is used to have a lower `CORPUS_SIZE` for testing/experimental purposes.

In [17]:
import glob
import nltk

path = '/content/'+extracted_file_name+'/*.txt'
files = glob.glob(path)

final_ip = "final_input.txt"
for name in files:
    with open(name) as f:
        with open(final_ip, "a") as f1:
            for line in f:
                f1.write(line)
                f1.write("\n")
            f1.write("\n")

DEMO_MODE = False #@param {type:"boolean"}

if DEMO_MODE:
  CORPUS_SIZE = 10000
else:
  CORPUS_SIZE = 100000000 #@param {type: "integer"}

print(CORPUS_SIZE)
  
!(head -n $CORPUS_SIZE final_input.txt) > subdataset.txt
!mv subdataset.txt dataset.txt

!head -n 20 dataset.txt

100000000
 source material is a  Wiki__Podcast__ioeffse  http www racistsandwich com episodes e whats so political about food photography w  Wiki__Celesta__szals  noche as a sjw liberal ill explain it for you in reply to the points you made as undeserving as this clusterfuck of a community is of my efforts im doing this partly because maybe some of you will listen because im a sucker for punishment and because it might be hilarious how you all reply also im bored why do these people ascribe malice to every little thing theres a problem with phrasing in social justice  Wiki__Discourse__fsief  at the moment coming from the top academic  Wiki__Discourse__fsief  to the bottom laymen who maybe read one article on social justice that one time theres a big disconnect not only in terminology but how to appropriately use that terminology like all things rooted in sociology and the  Wiki__Humanities__seiez  in general the  Wiki__Layman__foaobi  are given a media platform you dont see right leani

### Preprocess the dataset

In [0]:
regex_tokenizer = nltk.RegexpTokenizer("\w+")

def normalize_text(text):
  # lowercase text
  text = str(text).lower()
  # remove non-UTF
  text = text.encode("utf-8", "ignore").decode()
  # remove punctuation symbols
  text = " ".join(regex_tokenizer.tokenize(text))
  return text

def count_lines(filename):
  count = 0
  with open(filename) as fi:
    for line in fi:
      count += 1
  return count

In [19]:
RAW_DATA_FPATH = "/content/dataset.txt" #@param {type: "string"}
PRC_DATA_FPATH = "/content/proc_dataset.txt" #@param {type: "string"}

# apply normalization to the dataset
# this will take a minute or two

total_lines = count_lines(RAW_DATA_FPATH)
bar = Progbar(total_lines)

with open(RAW_DATA_FPATH,encoding="utf-8") as fi:
  with open(PRC_DATA_FPATH, "w",encoding="utf-8") as fo:
    for l in fi:
      fo.write(normalize_text(l)+"\n")
      bar.add(1)

!head -n 20 $PRC_DATA_FPATH

236118/236118 [==============================] - 18s 76us/step
source material is a wiki__podcast__ioeffse http www racistsandwich com episodes e whats so political about food photography w wiki__celesta__szals noche as a sjw liberal ill explain it for you in reply to the points you made as undeserving as this clusterfuck of a community is of my efforts im doing this partly because maybe some of you will listen because im a sucker for punishment and because it might be hilarious how you all reply also im bored why do these people ascribe malice to every little thing theres a problem with phrasing in social justice wiki__discourse__fsief at the moment coming from the top academic wiki__discourse__fsief to the bottom laymen who maybe read one article on social justice that one time theres a big disconnect not only in terminology but how to appropriately use that terminology like all things rooted in sociology and the wiki__humanities__seiez in general the wiki__layman__foaobi are given a

Here we build a dictionary of all the wiki words and count the occurences of each of them.

The dictionary is `wikis`.
We use RegularExpression parsing to get these statistics.

In [0]:
import re
wikis = dict()
with open(PRC_DATA_FPATH, "rt") as f:
    for i, line in enumerate(f):
        some_ = re.findall('wiki__[a-zA-Z0-9_]+', line, re.MULTILINE)   # Find all words that start with wiki__
        for wiki in some_:
            if(wiki in wikis.keys()):
                wikis[wiki] += 1
            else:
                wikis[wiki] = 1


In [24]:
len(wikis.keys()) # Around 78597 -> Number of unique wiki words

# Puth these in a dataframe for easy access and use.
import pandas as pd
df = pd.DataFrame(list(wikis.items()))
df = df.sort_values(1, ascending=False)
df.rename(columns = {0:'Wikified Words', 1:"Frequency"}, inplace = True)
df.head()

,Wikified Words,Frequency
41,wiki__liberal_arts__iablf,43117
3,wiki__humanities__seiez,42139
176,wiki__academic_term__sfssll,19185
100,wiki__whitelist__iazebq,15910
112,wiki__professor__iblaoo,11946


In [25]:
def most_frequent_wiki_words(n : int):
    return set(df.iloc[:n, 0])

most_frequent_n_wiki_words = most_frequent_wiki_words(WIKI_WORDS_IN_VOCAB)
print(most_frequent_n_wiki_words)

{'wiki__wikipedia__sofelef', 'wiki__fuck__sslslzz', 'wiki__doctor_of_philosophy__alls', 'wiki__direct_image_functor__eozzlff', 'wiki__student_financial_aid__iiloqaa', 'wiki__theres__ioisibsz', 'wiki__accountability__ibiqle', 'wiki__social_sciences__zblai', 'wiki__dude__eesleq', 'wiki__barack_obama__sefebb', 'wiki__essay__ioelf', 'wiki__humanities__seiez', 'wiki__shit__eazzi', 'wiki__gender_studies__saafq', 'wiki__wiki__ezasi', 'wiki__university__eilqf', 'wiki__whitelist__iazebq', 'wiki__sake__zasoo', 'wiki__twitter__qqaaial', 'wiki__scholarship__iaoeel', 'wiki__california__sfol', 'wiki__natural_science__eaaqo', 'wiki__tuition__zfqezo', 'wiki__ideology__lsfbq', 'wiki__donald_trump__fafazlz', 'wiki__academic_senate__ezieiiq', 'wiki__fine_art__qoeil', 'wiki__capitalism__sfib', 'wiki__standardized_test__zzsqli', 'wiki__literacy__iafsb', 'wiki__homework__ezbsfb', 'wiki__students__union__siqqoi', 'wiki__tutor__zqqqbs', 'wiki__stem_fields__efelbbe', 'wiki__interdisciplinarity__iszoi', 'wiki__

! IMPORTANT

This cell takes care of the wikification and de-wikification of the words.

If a wiki word occurs in `most_frequent_n_wiki_words`, then the wikification is changed as :

wiki__chicago__identifier -> wikichicagoidentifier

If a wiki word does NOT occur in `most_frequent_n_wiki_words`, then the wikification is changed as :

wiki__chicago__identifier -> chicago

wiki__illiois_tech__jargons ->illinois tech


In [0]:
'''
with open(PRC_DATA_FPATH, "rt") as f:
    data1 = f.read()
    data1 = data1.lower()
    words1 = data1.split()

new_set_of_words = list()
for i, word in enumerate(words1):

    if word in most_frequent_n_wiki_words:
        new_set_of_words.append(word.replace("_", ""))

    else:
        # if not most frequent : remove _ and wiki__ and __jargon
        if word[:6] == 'wiki__':
            new_set_of_words.append(word[6:word.rfind("__")].replace("_", " "))
        else:
            new_set_of_words.append(word)

    if i % 1000 == 0:
        new_set_of_words.append("\n")

# print(len(words1))
# print(len(new_set_of_words))

new_data = ' '.join([str(elem) for elem in new_set_of_words])
with open('new_dataset.txt', "w") as f:
    f.write(new_data)
'''

This cell is for the smart selection of the data from the corpus.

We only choose to include those lines that contain the most frequently occuring wiki-words in them. The other lines are ignored.

This is done so that the we can be sure that BERT is learning something meaninful in every line and not just being trainined on plain english.

In [0]:
with open(PRC_DATA_FPATH, "rt") as f:
    with open(SUBSAMPLED_DATA_FPATH, "a") as s:
        for i, line in enumerate(f):
            if(line == "\n"):
                s.write("\n")
                continue
            wiki_in_line = set(re.findall('wiki__[a-zA-Z0-9_]+', line, re.MULTILINE))
            if most_frequent_n_wiki_words & wiki_in_line:
                # if in most frequent : remove _'s from the word and
                # Put this in a new file
                    for j, word in enumerate(line.split(" ")):
                        if word in most_frequent_n_wiki_words:
                            s.write(word.replace("_", "")+ " ")

                        else:
                            # if not most frequent : remove _ and wiki__ and __jargon
                            if word[:6] == 'wiki__':
                                s.write(word[6:word.rfind("__")].replace("_", " ")+ " ")
                            else:
                                s.write(word + " ")

In [0]:
!head -n 21 /content/subsampled.txt


if trump really fucks things up then sure i could see more than just the left getting involved your people have been making every little thing seems like an apocalyptic event anything trump does is spun into something racism wikisexismzlibs homophobic anti semite anti islam etc the fact that you keep calling people nazism is hilariously pathetic the everyday wikiunitedstatesefeflso sees how your peaceful protests turn out how you silence anyone with an opposing calling them racist instead of having a thoughtful discussion the left is alienating itself into the fringe all that will be remaining are blue haired fat gender fluid highway blocking welfare dependent wikiliberalartsiablf degree educated flat out failures who do nothing for the greater good or even try to consider another perspective seriously look at the type of people who take part in all the protests and scream the loudest i would be so ashamed to even be remotely associated with that it is so crazy how the crap the left c

In [28]:
# Words to be added to the `vocab.txt`
for word in most_frequent_n_wiki_words:
    print(word.replace("_", ""))

wikiwikipediasofelef
wikifucksslslzz
wikidoctorofphilosophyalls
wikidirectimagefunctoreozzlff
wikistudentfinancialaidiiloqaa
wikitheresioisibsz
wikiaccountabilityibiqle
wikisocialscienceszblai
wikidudeeesleq
wikibarackobamasefebb
wikiessayioelf
wikihumanitiesseiez
wikishiteazzi
wikigenderstudiessaafq
wikiwikiezasi
wikiuniversityeilqf
wikiwhitelistiazebq
wikisakezasoo
wikitwitterqqaaial
wikischolarshipiaoeel
wikicaliforniasfol
wikinaturalscienceeaaqo
wikituitionzfqezo
wikiideologylsfbq
wikidonaldtrumpfafazlz
wikiacademicsenateezieiiq
wikifineartqoeil
wikicapitalismsfib
wikistandardizedtestzzsqli
wikiliteracyiafsb
wikihomeworkezbsfb
wikistudentsunionsiqqoi
wikitutorzqqqbs
wikistemfieldsefelbbe
wikiinterdisciplinarityiszoi
wikilolzaaelq
wikistudentloanfbfqqo
wikiportablenetworkgraphicszfeob
wikimathematicsiaaei
wikirhetoriczsffl
wikiunemploymenteilfi
wikicurriculumflqqae
wikiblogeebfs
wikiaccountancyzsqe
wikiseminarbllooo
wikiidiotisoof
wikiundergraduateeducationzilzaq
wikijournalismisqza

### Create Pre-training Data

Before doing this, we need to make sure to download the vocab.txt from the [Google BERT Repository](https://github.com/google-research/bert#pre-trained-models). In this vocab.txt, we need to replace the required number of `[UNUSED_N]` words with these frequently occuring wiki-words.

It is essential that this is done before we create pre-training data.

Also make sure that the name of this vocab file is `vocab.txt`

In [0]:
!mkdir ./shards_new

# Split the data file into shards so that the data chunks fit in the RAM.
# Feel free to modify the arguments as per requirement. 
# More information about the command here : http://man7.org/linux/man-pages/man1/split.1.html
!split -a 4 -l 250000 -d $SUBSAMPLED_DATA_FPATH ./shards_new/shard_
!ls ./shards_new/

shard_0000


In [0]:
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param
MAX_PREDICTIONS = 20 #@param {type:"integer"}
DO_LOWER_CASE = True #@param {type:"boolean"}
PROCESSES = 2 #@param {type:"integer"}
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}
VOC_FNAME = "/content/vocab.txt" # This is the same vocab file talked about earlier

In [0]:
XARGS_CMD = ("ls ./shards_new/ | "
             "xargs -n 1 -P {} -I{} "
             "python3 bert/create_pretraining_data.py "
             "--input_file=./shards_new/{} "
             "--output_file={}/{}.tfrecord "
             "--vocab_file={} "
             "--do_lower_case={} "
             "--max_predictions_per_seq={} "
             "--max_seq_length={} "
             "--masked_lm_prob={} "
             "--random_seed=34 "
             "--dupe_factor=5")

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}', PRETRAINING_DIR, '{}', 
                             VOC_FNAME, DO_LOWER_CASE, 
                             MAX_PREDICTIONS, MAX_SEQ_LENGTH, MASKED_LM_PROB)

In [0]:
tf.gfile.MkDir(PRETRAINING_DIR)
!$XARGS_CMD



W0429 22:47:42.883340 140629406877568 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0429 22:47:42.883568 140629406877568 module_wrapper.py:139] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0429 22:47:42.883748 140629406877568 module_wrapper.py:139] From /content/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0429 22:47:42.966245 140629406877568 module_wrapper.py:139] From bert/create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0429 22:47:42.969016 140629406877568 module_wrapper.py:139] From bert/create_pretraining_data.py:446: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:*** Reading from input files ***

### Preparing to PreTrain

BEFORE EXECUTING THESE,


1.   We need to upload the weight and config files to a Google Cloud Storage Bucket from where this BERT model will get and save its data.
2.   To do this, create a new Google Cloud Storage Bucket
3.   Then upload all the files we downloaded from the Google BERT Pre-trained model repository to the Cloud Bucket in a folder whose name is the same as that in `MODEL_DIR` variable below.
4.   Replace the `vocab.txt` file on the Cloud Bucket with the `vocab.txt` file we modified and used to build the pre-training data. 
5.   Make sure that the `BUCKET_NAME` variable below matches the exact name of the Google Cloud Bucket we created, and the name of the vocab file matches the one we uploaded.




In [0]:
BUCKET_NAME = "<bucket_name>" #@param {type:"string"}
MODEL_DIR = "<dir_name>" #@param {type:"string"}
PRETRAINING_DIR = "<dir_name>" #@param {type:"string"}
VOC_FNAME = "vocab.txt" #@param {type:"string"}

tf.gfile.MkDir(MODEL_DIR)

if not BUCKET_NAME:
  log.warning("WARNING: BUCKET_NAME is not set. "
              "You will not be able to train the model.")

In [0]:
# Copy the pretraining data -> .tfrecord files to the GCS Bucket
if BUCKET_NAME:
  !gsutil -m cp -r $PRETRAINING_DIR gs://$BUCKET_NAME

Copying file://pretraining_data/shard_0000.tfrecord [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1/1 files][315.3 MiB/315.3 MiB] 100% Done                                    
Operation completed over 1 objects/315.3 MiB.                                    


### Pre-train

In [0]:
# Input data pipeline config
TRAIN_BATCH_SIZE = 128 #@param {type:"integer"}
MAX_PREDICTIONS = 20 #@param {type:"integer"}
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param

# Training procedure config
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 1e-4
TRAIN_STEPS = 10000 #@param {type:"integer"}
SAVE_CHECKPOINTS_STEPS = 2500 #@param {type:"integer"}
NUM_TPU_CORES = 8

if BUCKET_NAME:
  BUCKET_PATH = "gs://{}".format(BUCKET_NAME)
else:
  BUCKET_PATH = "."

BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, MODEL_DIR)
DATA_GCS_DIR = "{}/{}".format(BUCKET_PATH, PRETRAINING_DIR)

VOCAB_FILE = os.path.join(BERT_GCS_DIR, VOC_FNAME)
CONFIG_FILE = os.path.join(BERT_GCS_DIR, "bert_config.json")

# If we want to start off from Google's Checkpoint :
INIT_CHECKPOINT = os.path.join(BERT_GCS_DIR, "bert_model.ckpt")

# If we want to start off from our own latest checkpoint
# INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR)


bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.gfile.Glob(os.path.join(DATA_GCS_DIR,'*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))
log.info("BERT_GCS_DIR {}".format(BERT_GCS_DIR))
log.info("Using {} vocab file".format(VOCAB_FILE))

2020-04-29 23:12:42,740 :  From /content/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

2020-04-29 23:12:44,176 :  Using checkpoint: gs://bert_apr/bert_model_final/bert_model.ckpt
2020-04-29 23:12:44,179 :  Using 1 data shards
2020-04-29 23:12:44,182 :  BERT_GCS_DIR gs://bert_apr/bert_model_final
2020-04-29 23:12:44,184 :  Using gs://bert_apr/bert_model_final/vocab.txt vocab file


In [0]:
# Verifying if the correct vocab file is being used in the pre-training,
# Reading the vocab.txt file on the cloud bucket.

!gsutil cat $VOCAB_FILE | head -107

[PAD]
wikiliberalartsiablf
wikihumanitiesseiez
wikiacademictermsfssll
wikiwhitelistiazebq
wikiprofessoriblaoo
wikisocialscienceszblai
wikiredditeazqoos
wikisocialmediasaqllfz
wikiblogeebfs
wikiunitedstatesefeflso
wikicurriculumflqqae
wikigraduateschoolefbbza
wikigraphicsinterchangeformatizloz
wikiportablenetworkgraphicszfeob
wikijpegibooq
wikidirectimagefunctoreozzlff
wikimathematicsiaaei
wikituitionzfqezo
wikilolzaaelq
wikidoctorofphilosophyalls
wikiundergraduateeducationzilzaq
wikishiteazzi
wikitwitterqqaaial
wikifascismiiosf
wikihomeworkezbsfb
wikistudentfinancialaidiiloqaa
wikiinternzqazzi
wikidissertationeoozis
wikilaboureconomicsiaila
wikicriticalthinkingzlbalz
wikipermalinkalaqzb
wikifacebooklszqela
wikiwikipediasofelef
wikihypertexttransferprotocoloversecuresocketlayeriesab
wikiemailqlea
wikiuniversityeilqf
wikischolarshipiaoeel
wikidudeeesleq
wikitheresioisibsz
wikisexismzlibs
wikiharvarduniversityiafzbsoi
wikiinterdisciplinarityiszoi
wikiyoutubeeszflbb
wikiartsintegrationqqeb

In [0]:
model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=TRAIN_STEPS,
      num_warmup_steps=10,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=True)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)

2020-04-29 23:13:05,718 :  Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fdb90c8dae8>) includes params argument, but params are not passed to Estimator.
2020-04-29 23:13:05,722 :  Using config: {'_model_dir': 'gs://bert_apr/bert_model_final', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.39.244.154:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdb35482470>, '

In [0]:
estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

2020-04-29 23:13:14,549 :  Querying Tensorflow master (grpc://10.39.244.154:8470) for TPU system metadata.
2020-04-29 23:13:14,580 :  Found TPU system:
2020-04-29 23:13:14,581 :  *** Num TPU Cores: 8
2020-04-29 23:13:14,582 :  *** Num TPU Workers: 1
2020-04-29 23:13:14,585 :  *** Num TPU Cores Per Worker: 8
2020-04-29 23:13:14,587 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 15751215702725798585)
2020-04-29 23:13:14,589 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 17154297229405720259)
2020-04-29 23:13:14,591 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 708830170763276420)
2020-04-29 23:13:14,592 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 12685520322608514198)
2020-04-29 23:13:14,594 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device

2020-04-29 23:13:17,408 :  From bert/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

2020-04-29 23:13:17,471 :  From bert/modeling.py:358: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2020-04-29 23:13:17,497 :  From bert/modeling.py:671: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.Dense instead.
2020-04-29 23:13:17,502 :  From /tensorflow-1.15.2/python3.6/tensorflow_core/python/layers/core.py:187: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
2020-04-29 23:13:20,291 :  From /content/bert/run_pretra

In [0]:
estimator.train(input_fn=train_input_fn, max_steps=5*TRAIN_STEPS)

2020-04-29 23:33:54,505 :  Calling model_fn.
2020-04-29 23:33:54,555 :  Entity <function input_fn_builder.<locals>.input_fn.<locals>.<lambda> at 0x7fda75449ae8> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: module 'gast' has no attribute 'Str'
2020-04-29 23:33:54,593 :  Found small feature: next_sentence_labels [16, 1]
2020-04-29 23:33:54,596 :  Found small feature: next_sentence_labels [16, 1]
2020-04-29 23:33:54,601 :  Found small feature: next_sentence_labels [16, 1]
2020-04-29 23:33:54,605 :  Found small feature: next_sentence_labels [16, 1]
2020-04-29 23:33:54,610 :  Found small feature: next_sentence_labels [16, 1]
2020-04-29 23:33:54,614 :  Found small feature: next_sentence_labels [16, 1]
2020-04-29 23:33:54,619 :  Found small feature: next_sentence_labels [16, 1]
2020-04-29 23:33:54,624 :  Found small featu

2020-04-29 23:34:02,126 :  **** Trainable Variables ****
2020-04-29 23:34:02,142 :    name = bert/embeddings/word_embeddings:0, shape = (28996, 768), *INIT_FROM_CKPT*
2020-04-29 23:34:02,149 :    name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
2020-04-29 23:34:02,151 :    name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
2020-04-29 23:34:02,155 :    name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
2020-04-29 23:34:02,157 :    name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
2020-04-29 23:34:02,159 :    name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
2020-04-29 23:34:02,161 :    name = bert/encoder/layer_0/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
2020-04-29 23:34:02,164 :    name = bert/encoder/layer_0/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
2020-04-29 23:34:02,169 :    name =

In [0]:
print("DONE")

DONE
